# Basic tutorial of CoolPrompt
*Actual for 1.1.0+ version*

## Quick Run

In [3]:
from coolprompt.assistant import PromptTuner

prompt_tuner = PromptTuner()

prompt_tuner.run('Write an essay about autumn')

In [4]:
print(prompt_tuner.final_prompt)


You are an expert writer and seasonal observer tasked with composing a rich, well-structured, and vividly descriptive essay on the theme of autumn. Your goal is to create a coherent, engaging, and insightful piece that captures the essence of the season through sensory detail, natural observation, and cultural reflection.

Begin by describing the key physical and environmental characteristics of autumn: the transformation of foliage, the gradual cooling of temperatures, the crispness of the air, and the quiet rhythm of nature as leaves fall. Use specific, evocative language to convey sight, sound, smell, and touch—such as the rustle of dry leaves, the golden light filtering through trees, or the earthy scent of damp soil after rain.

Integrate cultural and human elements of autumn, including harvest festivals (e.g., Thanksgiving, Halloween, harvest fairs), traditional foods (e.g., apples, pumpkins, squash), and seasonal rituals that reflect humanity’s connection to nature. Reflect on 

### What happened step-by-step
1. We initialized a CoolPrompt Tuner with default large language model: qwen3-4B-instruct
2. We input a start prompt that should be modified
3. CoolPrompt auto-detected preferred task and metric for prompt optimization and evaluation
4. CoolPrompt generated automatically the dataset with target labels and split into train and test samples
5. The default prompt optimizer HyPE suggested the optimized prompt
6. CoolPrompt outputs the results:
   - Metric scores of optimization efficiency
   - The final prompt
   - The interpretation of prompt optimization

### That is a simple approach how to optimize prompts. Let's try to configure CoolPrompt Tuner

## Setup a CoolPrompt Tuner

### 1. LLM Choice
The framework is a model-agnostic, you can use proprietary, open-source or custom LLMs with different interfaces via LangChain compatibility. 

#### Check [the full list of provider interfaces](https://python.langchain.com/docs/integrations/llms/).

Default LLM in CoolPrompt is Qwen/Qwen3-4B-Instruct-2507 and it a same can be defined as:

```python
llm = HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen3-4B-Instruct-2507",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 4000,
        "temperature": 0.01,
        "do_sample": False,
        "return_full_text": False,
    }
)
target_model = ChatHuggingFace(llm=llm)

prompt_tuner = PromptTuner(target_model=target_model)
```


You can use other popular interfaces as:

#### Ollama
For rapid experiments with low resources

In [ ]:
# Before the run command serve the model with ollama
from langchain_ollama.llms import OllamaLLM

my_model = OllamaLLM(
    model="qwen2.5-coder:32b"
)
prompt_tuner = PromptTuner(target_model=my_model)

#### VLLM
As a production ready solution

In [ ]:
from langchain_community.llms import VLLM

my_model = VLLM(
    model="Qwen/Qwen3-4B-Instruct-2507",
    trust_remote_code=True,
    dtype='bfloat16',
)

prompt_tuner = PromptTuner(target_model=my_model)

#### ChatOpenAI
For OpenAI and OpenAI compatible models

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

my_model = ChatOpenAI(
    model="paste model_name",
    base_url="paste base_url",
    openai_api_key="paste api_key",
    temperature=0.01,
    max_tokens=4000,
)

prompt_tuner = PromptTuner(target_model=my_model)

### 2. System Model
Argument `system_model` defines a core in a **PromptAssistant** module.

**PromptAssistant** is responsible for:
- Definition automatically a task type and a metric for the specific start prompt.
- Generates a syntetic dataset and target labels (when the real data is not provided).
- Provides an optimization feedback.

By default a `system_model` is a defined the same as a `target_model`. It could be set with a different llm in code below:

In [42]:
from langchain_community.llms import VLLM

target_model = VLLM(
    model="Qwen/Qwen3-4B-Instruct-2507"
)
system_model = VLLM(
    model="Qwen/Qwen3-30B-A3B-Instruct-2507"
)

prompt_tuner = PromptTuner(
    target_model=target_model,
    system_model=system_model
)

#### IMPORTANT NOTE: `system_model` needs to be a confident instructional llm that can generate a structual output.

### 3. Task and metrics
CoolPrompt supports 2 task types and a set of metrics: 
- `classification` - a common classification
    - `accuracy`
    - `f1` (f1-macro)
- `generation` - a general new text generation
    - `bleu`
    - `rouge`
    - `meteor`
    - `bertscore`
    - `geval` (experimental, one metric: *textual accuracy*)

For each run you can define as follows:

In [ ]:
prompt_tuner.run(
    start_prompt="Classify a sentence sentiment",
    task="classification",
    metric="f1",
)

prompt_tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    metric="rouge",
)

### 4. Setup a dataset configuration

#### 4.1 Input a real dataset

In [41]:
from langchain_openai.chat_models import ChatOpenAI

my_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key="key",
    temperature=0,
    max_tokens=3500,
)

prompt_tuner = PromptTuner(target_model=my_model)

[2025-10-12 12:13:30,157] [INFO] [assistant.__init__] - Validating the target model
[2025-10-12 12:13:30,158] [INFO] [assistant.__init__] - PromptTuner successfully initialized


In [40]:
from datasets import load_dataset

sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:30]
class_targets = sst2['train']['label'][:30]

prompt_tuner.run(
    start_prompt="Classify sentence sentiment positive or negative",
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="f1"
)

print(prompt_tuner.final_prompt)

[2025-10-12 12:12:49,368] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-10-12 12:12:50,423] [INFO] [evaluator.__init__] - Evaluator successfully initialized with f1 metric
/venv/main/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1949: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
[2025-10-12 12:12:51,334] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-10-12 12:12:51,335] [INFO] [assistant.run] - Method: hype, Task: classification
[2025-10-12 12:12:51,336] [INFO] [assistant.run] - Metric: f1, Validation size: 0.25
[2025-10-12 12:12:51,337] [INFO] [assistant.run] - Dataset: 30 samples
[2025-10-12 12:12:51,338] [INFO] [assist


Train a machine learning model to classify the sentiment of sentences as either positive or negative. Use a dataset of labeled sentences where each sentence is associated with its sentiment label. Aim to create a classifier that can accurately predict the sentiment of new, unseen sentences based on the patterns and features in the training data.



In [37]:
samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:30]
gen_targets = samsum['train']['summary'][:30]

prompt_tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
)

print(prompt_tuner.final_prompt)

[2025-10-12 12:11:45,024] [INFO] [assistant.run] - Validating args for PromptTuner running
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[2025-10-12 12:11:46,060] [INFO] [evaluator.__init__] - Evaluator successfully initialized with meteor metric
/venv/main/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1949: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
[2025-10-12 12:11


Given a text, create a concise summary by extracting the most important information and presenting it in a condensed form.



#### 4.2 Define a split strategy

`validation_size` - a ratio for train-valid split

In [ ]:
prompt_tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="bertscore",
    validation_size=0.4
)

`train_as_test` - assign a test set as a train, `validation_size` will be ignored

In [ ]:
prompt_tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="bertscore",
    train_as_test=True
)

### 5. Choosing an optimizer

#### HyPE

Provides the prompt refactoring workflow: the initial prompt is injected into special predetermined by our researches query template. 

This optimizer requires only one query to the LLM, so it surely can be used as a fast and simple tool to make your prompt better.

In [24]:
prompt_tuner.run(
    start_prompt="Summarize the text",
    method='hype'
)

print(prompt_tuner.final_prompt)

[2025-10-12 12:01:33,328] [INFO] [detector.generate] - Detecting the task by query
/venv/main/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1949: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
[2025-10-12 12:01:33,849] [INFO] [detector.generate] - Task defined as generation
[2025-10-12 12:01:33,850] [INFO] [assistant.run] - Validating args for PromptTuner running
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   P


Given a text, create a concise summary by extracting the most important information. Ensure the summary is brief yet informative for the user.



#### ReflectivePrompt

This method is based on the idea of Reflective Evolution and text-based gradients. It implements short-term and long-term reflections to provide some clarifications and make crossover and mutation operations more precise and effective.

ReflectivePrompt has 2 arguments:
- population_size (default: 10)
- num_epochs (default: 5)


In [25]:
prompt_tuner.run(
    start_prompt="Summarize the text",
    method='reflective',
    num_epochs=2
)

[2025-10-12 12:02:56,733] [INFO] [detector.generate] - Detecting the task by query
/venv/main/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1949: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
[2025-10-12 12:02:57,194] [INFO] [detector.generate] - Task defined as generation
[2025-10-12 12:02:57,194] [INFO] [assistant.run] - Validating args for PromptTuner running
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   P

#### DistillPrompt

This method is based on different prompt-transformation methods that will use LLM to effectively examine the search area and find the best prompt variations for given task.

DistillPrompt has 1 argument:
- num_epochs (default: 5)

In [27]:
prompt_tuner.run(
    start_prompt="Summarize the text",
    method='distill',
    num_epochs=2
)

[2025-10-12 12:08:01,668] [INFO] [detector.generate] - Detecting the task by query
/venv/main/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1949: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
[2025-10-12 12:08:02,166] [INFO] [detector.generate] - Task defined as generation
[2025-10-12 12:08:02,166] [INFO] [assistant.run] - Validating args for PromptTuner running
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   P

## Framework Results

#### Metrics before-after

In [29]:
print('Before optimization:', prompt_tuner.init_metric)
print('After optimization:', prompt_tuner.final_metric)

Before optimization: 0.2138408105858377
After optimization: 0.7006453459110112


#### Prompts before-after

In [31]:
print('Before optimization:', prompt_tuner.init_prompt)
print('After optimization:', prompt_tuner.final_prompt)

Before optimization: Summarize the text
After optimization: Summarize text by identifying main ideas and key points concisely for clarity and coherence.


#### Generated dataset

In [32]:
print(prompt_tuner.synthetic_dataset)
print(prompt_tuner.synthetic_target)

['A new study shows that regular exercise can improve mental health and reduce the risk of depression.', 'The latest research indicates that eating a balanced diet can lead to better overall health and well-being.', 'Scientists have discovered a new treatment for a common type of cancer that shows promising results in clinical trials.', 'A recent report highlights the importance of sleep for cognitive function and overall productivity.', 'Experts suggest that practicing mindfulness meditation can reduce stress and improve mental clarity.', 'A study found that spending time in nature can have a positive impact on mental health and well-being.', 'Researchers have developed a new technology that could revolutionize renewable energy production.', 'A report highlights the importance of regular physical activity in maintaining a healthy lifestyle and preventing chronic diseases.', "Scientists have identified a gene that may play a role in the development of Alzheimer's disease.", 'Recent stu

#### Optimization feedback

In [34]:
print('Feedback:', prompt_tuner.assistant_feedback)

Feedback: Your original prompt was quite vague and lacked specificity. We enhanced it by adding clarity and focus on the task by specifying to summarize text by identifying main ideas and key points concisely. This helps the AI understand the exact requirements and deliver a more targeted response. The revised prompt emphasizes the importance of conciseness for clarity and coherence, which guides the AI to provide a more structured and coherent summary. Key advice: Always aim for specificity in your prompts to guide the AI effectively.


# LLM as judge mertic #


There are 4 available templates for llm as judge metric:

- Accuracy: The answer is factually correct and contains no mistakes. It uses the right terms and numbers and does not introduce wrong information.

- Coherence: The answer is well structured and easy to follow. 
Ideas are in a logical order and there are no contradictions.

- Fluency: The answer is written in natural, correct language. 
Grammar, vocabulary and phrasing are clear and do not make the text hard to read.

- Relevance: The answer directly responds to the request and stays on topic. 
It does not add unnecessary or unrelated details. 


In [1]:
from datasets import load_dataset
from coolprompt import PromptTuner
from langchain_ollama import ChatOllama

target_model = ChatOllama(model="qwen2.5:1.5b-instruct")
system_model = ChatOllama(model="qwen2.5:3b-instruct")

tuner = PromptTuner(target_model=target_model, system_model=system_model)


samsum = load_dataset("knkarthick/samsum")
dataset = samsum["train"]["dialogue"][:40]
targets = samsum["train"]["summary"][:40]


result = tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=dataset,
    target=targets,
    method="hype",
    metric="llm_as_judge",
    llm_as_judge_criteria="relevance",
    generate_num_samples=20,
    verbose=2,
)

# You can use several criteria, and the final score will be the mean of them.

result2 = tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=dataset,
    target=targets,
    method="hype",
    metric="llm_as_judge",
    llm_as_judge_criteria=["relevance", "coherence"],
    generate_num_samples=20,
    verbose=2,
)



# You can use custom criteria in llm as judge

custom = {
    "creativity": """You will be given a response to a question.
    Rate the creativity on a scale from 1 to {metric_ceil}.
    Return ONLY a single number.

    Source: {request}
    Response: {response}

    Creativity score (number only):"""
}

result3 = tuner.run(
    start_prompt="",
    task="generation",
    dataset=dataset,
    target=targets,
    method="hype",
    metric="llm_as_judge",
    llm_as_judge_criteria=["creativity"],
    llm_as_judge_custom_templates=custom,
    generate_num_samples=20,
    verbose=2,
)

[2025-11-19 16:52:25,911] [INFO] [assistant.__init__] - Validating the target model
[2025-11-19 16:52:25,911] [INFO] [assistant.__init__] - Validating the system model
[2025-11-19 16:52:25,912] [INFO] [assistant.__init__] - PromptTuner successfully initialized
[2025-11-19 16:52:54,452] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-11-19 16:52:54,453] [INFO] [evaluator.__init__] - Evaluator successfully initialized with llm_as_judge metric
[2025-11-19 16:53:04,941] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-11-19 16:53:04,942] [INFO] [assistant.run] - Method: hype, Task: generation
[2025-11-19 16:53:04,943] [INFO] [assistant.run] - Metric: llm_as_judge, Validation size: 0.25
[2025-11-19 16:53:04,943] [INFO] [assistant.run] - Dataset: 40 samples
[2025-11-19 16:53:04,943] [INFO] [assistant.run] - Target: 40 samples
[2025-11-19 16:53:04,944] [INFO] [hype.hype_optimizer] - Running HyPE optimization...
[2025-11-19 16:53:04,944] [DEBUG] [hype.

# Deepeval GEval metric #

GEval is an LLM-as-a-judge metric that checks whether a generated answer matches the expected output according to natural-language instructions. 

You can pass either a single `geval_criteria` string (as in the example below) or a list of explicit `evaluation_steps`, and optionally select which fields the judge sees via `evaluation_params` (default INPUT, ACTUAL_OUTPUT, EXPECTED_OUTPUT).

You can use Ollama model or ChatOpenAI model

In [4]:
from coolprompt.assistant import PromptTuner
from langchain_ollama import ChatOllama
from datasets import load_dataset

model = ChatOllama(model="qwen2.5:1.5b-instruct")

tuner = PromptTuner(target_model=model, system_model=model)

start_prompt = "Summarize the text"


samsum = load_dataset("knkarthick/samsum")
dataset = samsum["train"]["dialogue"][:40]
targets = samsum["train"]["summary"][:40]

geval_criteria = (
    "Evaluate whether the assistant's summary is an accurate, concise, and coherent "
    "paraphrase of the original text. The summary should capture all key facts and "
    "important events from the expected output, avoid adding new information that is "
    "not supported by the original text, and omit minor irrelevant details. "
)
res = tuner.run(
    start_prompt=start_prompt,
    task="generation",
    dataset=dataset,
    target=targets,
    method="hype",
    metric="geval",
    geval_criteria=geval_criteria,
    verbose=2
)

print("Initial GEval score:", tuner.init_metric)
print("Final GEval score:", tuner.final_metric)
print("Final prompt", res)

[2025-11-21 01:01:13,640] [INFO] [assistant.__init__] - Validating the target model
[2025-11-21 01:01:13,641] [INFO] [assistant.__init__] - PromptTuner successfully initialized
[2025-11-21 01:01:13,641] [INFO] [assistant.__init__] - PromptTuner successfully initialized


[2025-11-21 01:01:19,574] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-11-21 01:01:19,575] [INFO] [evaluator.__init__] - Evaluator successfully initialized with geval metric
[2025-11-21 01:01:19,575] [INFO] [evaluator.__init__] - Evaluator successfully initialized with geval metric
[2025-11-21 01:01:20,566] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-11-21 01:01:20,567] [INFO] [assistant.run] - Method: hype, Task: generation
[2025-11-21 01:01:20,568] [INFO] [assistant.run] - Metric: geval, Validation size: 0.25
[2025-11-21 01:01:20,569] [INFO] [assistant.run] - Dataset: 40 samples
[2025-11-21 01:01:20,569] [INFO] [assistant.run] - Target: 40 samples
[2025-11-21 01:01:20,570] [INFO] [hype.hype_optimizer] - Running HyPE optimization...
[2025-11-21 01:01:20,570] [DEBUG] [hype.hype_optimizer] - Start prompt:
Summarize the text
[2025-11-21 01:01:20,566] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-11-21 01:01:20,567] [

Initial GEval score: 0.44000000000000006
Final GEval score: 0.52
Final prompt Please provide the text that needs summarization, including any code snippets or specific details you would like included in the summary. I will then generate a concise summary capturing the key points and essential information from the provided content.


Using evaluation steps example

In [ ]:
import os
from getpass import getpass

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    os.environ["OPENAI_API_KEY"] = "your key"

from langchain_openai import ChatOpenAI
from datasets import load_dataset
from coolprompt.assistant import PromptTuner

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

samsum = load_dataset("knkarthick/samsum")
dataset = samsum["train"]["dialogue"][:40]
targets = samsum["train"]["summary"][:40]

geval_steps = [
    "1. Compare the assistant's summary with the reference and list the key facts each one mentions.",
    "2. Check whether the assistant introduces any unsupported information or misses essential events.",
    "3. Decide if the assistant's summary is concise and coherent while covering all required details.",
]

tuner = PromptTuner(target_model=model, system_model=model)

final_prompt = tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=dataset,
    target=targets,
    method="hype",
    metric="geval",
    geval_evaluation_steps=geval_steps,
    verbose=2,
 )

print("Initial GEval score:", tuner.init_metric)
print("Final GEval score:", tuner.final_metric)
print("Final prompt:", final_prompt)

[2025-11-21 19:06:05,048] [INFO] [assistant.__init__] - Validating the target model
[2025-11-21 19:06:05,049] [INFO] [assistant.__init__] - PromptTuner successfully initialized
[2025-11-21 19:06:05,050] [INFO] [assistant.run] - Validating args for PromptTuner running
[2025-11-21 19:06:05,050] [INFO] [evaluator.__init__] - Evaluator successfully initialized with geval metric
/home/asd480/autoprompt/CoolPrompt/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1963: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
[2025-11-21 19:06:07,421] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-11-21 19:06:07,422] [INFO] [assistant.run] - Method: hype, Task: g

Initial GEval score: 0.24
Final GEval score: 0.27999999999999997
Final prompt: 
Given a text, create a concise summary that captures the main points effectively. Ensure that the summary is brief yet informative, highlighting the key information from the original text.

